# Genome Sequencing

In [ ]:
import math
from collections import defaultdict
import urllib.request


---
### **1A**
Loading the read pairs from the data file (each read pair is in a separate line and each pair has been
separated by a “|” sign), and removing the lines corresponding to the metadata (the first two lines).
Afterwards, constructing the paired de Bruijn graph from this data. Printing the upper left 20 x 20 part of the
de Bruijn adjacency matrix constructed.

In [ ]:
# Retrieve the data using the URL and traverse the lines
# Convert type bytes-formated lines into strings
# Detect the input and output lines, keep the pairs into a list in the form of tuples 

url = "https://bioinformaticsalgorithms.com/data/extradatasets/assembly/string_reconstruction_from_read_pairs.txt"
data = urllib.request.urlopen(url)

pairs = []
output = ''

is_input=True
for i, line in enumerate(data):
  line = line.decode("utf-8").strip()
  if i == 0:
    continue
  if i == 1:
    values = line.split(' ')
    K = values[0]
    D = values[1]
    continue
  elif(line=='Output'):
    is_input = False
    continue
  elif is_input:
    pair = line.split('|')
    pair = (pair[0], pair[1])
    pairs.append(pair)
  else:
    output = line

In [ ]:
# After obtaining a list of pair tuples, create the adjacency matrix
# Create a set of nodes and save all edges to a list for further usage
res = []
nodes = set()
for pair in pairs:
    prefix = (pair[0][:-1], pair[1][:-1])
    suffix = (pair[0][1:], pair[1][1:])
    res.append((prefix, suffix))
    nodes.add(prefix)
    nodes.add(suffix)

# order the nodes with alphabetical order
ordered_nodes = sorted(list(nodes))

# create adjacency matrix in the size of nodes
adjacency = {}
for i, node_one in enumerate(ordered_nodes):
  adjacency[i] = [0 for x in range(len(ordered_nodes))]

# For the previously saved pairs, make the value in adjacency matrix 1 as there is an edge from prefix to suffix
for (pre, suf) in res:
  adjacency[ordered_nodes.index(pre)][ordered_nodes.index(suf)] =1

In [ ]:
# Print the left-top 20x20 part of the created adjacency matrix

print("Bruijn Adjacency Matrix")
for i in range(0,20):
  print(" ".join([str(x) for x in adjacency[i][:20]]))


Bruijn Adjacency Matrix
0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0




---


### **1B**
Finding the unique Eulerian path in the de Bruijn graph and reconstructing the associated genome
sequence accordingly. Printing the length of the final reconstructed genome.

In [ ]:
# Before searching for a cycle in adjacency matrix, find and connect the starting and ending nodes
# the starting node is the node where there exists 0 coming edges to it
# the end node is the node where there exists 0 going edges from it
# Detect the starting and ending nodes from the adjacency matrix
start_node = -1
end_node = -1
for i in range(0, len(ordered_nodes)):
  outgoing = sum(adjacency[i])
  incoming = 0
  for j in range(0, len(ordered_nodes)):
    incoming += adjacency[j][i]
  if incoming == 0:
    start_node = i
  if outgoing == 0:
    end_node = i

# To create a cycle, add an edge from the end node to the start node
print(start_node, end_node)
adjacency[end_node][start_node] = 1

2846 2593


In [ ]:
# Using the Hierholzer's Algorithm to find the Euler path. see: https://algorithms.discrete.ma.tum.de/graph-algorithms/hierholzer/index_en.html

def find_eulerian_path(edges_matrix , start_node):
  result = []

  # I start the algorithm with the starting node I have determined
  node = start_node
  path = [node]


  while True:
    if path == []:
      break

		# If there are unvisited edges from the node, use the first edge, remove the used edge and jump to next node
    if edges_matrix[node]:
      path.append(node)                             # add this node to path
      next_node = edges_matrix[node][0]             # jump to next node
      edges_matrix[node] = edges_matrix[node][1:]   # remove the used edge
      node = next_node

    # If there are not unvisited edges from the node, then go backwards
    else:
      next_node = path[-1]  # jump to previous node
      result.append(node)   # add this node to result
      node = next_node      # jump to previous node
      path = path[:-1]      # remove one step from the path
  
  # reverse the result before returning, remove the last index because it is the same with the first one
  result.reverse()
  return result[:-1]


In [ ]:
# Using the adjacency matrix, create a map of edges to be used in the euler path algorithm
edges = {}
for i in range(len(ordered_nodes)):
  l = [j for j in range(len(ordered_nodes)) if adjacency[i][j]==1]
  edges[i]=l

euler_path = find_eulerian_path(edges, start_node)

In [ ]:
# Reconstruct the genome using the Euler path calculated

def reconstruct(path, d):
  genome = ''

  # Start with the first node in the path, find the pair corresponding to that node from ordered_nodes
  initial_pair = ordered_nodes[path[0]]
  
  # Add the first element of the initial tuple to the genome as a whole
  # Then for each remaining node in the path until the D+1th, Find the pair corresponding to that node from ordered_nodes
  # Add the last gene of the first element of the tuple to the genome
  genome += initial_pair[0]
  for node in path[1: d + 2]:
    pair = ordered_nodes[node]
    genome += pair[0][-1]

  # Do the same with the second elements of the tuples now
  # This time consume the whole path 
  genome += initial_pair[1]
  for node in path[1:]:
    pair = ordered_nodes[node]
    genome += pair[1][-1]
  
  # return the reconstructed genome
  return genome

genome = reconstruct(euler_path, int(D))

In [ ]:
genome == output

True



---


### **1C**
Printing the first 200 and last 200 characters of the reconstructed genome string.

In [ ]:
print(genome[:200])

GAAAGGTACAAATACTGGCGACCTCGCTGTTCGACACTTCATCACTGCTCCGGGGCGCTCAGGAGGGACGGTTCCCTGTACCATTGGAAGTCAATAGTCTAAGGTACAAAGAGAAGACCCGACCCGACAGAGGGGGTTCTGCGCCGGGTTTCGAGCTTGTAACCCCCCAGAGAATTAGATCCACCGTCTGTGTGGACAAA


In [ ]:
print(genome[-200:])

GGGCAAATTATCAGCGTACAATTCCCAGATATATAGGCGGCGAGAAAAGCTTCAAAAGACTTAATTTACTAGCCTCCTACAAACTCTAGATGAGGATTGGCTCTTGATGCTAGCGTTTTCATTTTCCATTACAAGACATTAGGCTGATAATTGCAGAGATTGGCGGCGTAGACTGACAGTCGCGATCAATCTGCGTGTTA


In [ ]:
print(genome[:200] == output[:200])
print(genome[-200:] == output[-200:])

True
True
